In [ ]:
import pandas as pd
import os
import numpy as np

# import Data

In [ ]:
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path,'train.csv')
test_file_path= os.path.join(processed_data_path,'test.csv')

In [ ]:
train_df = pd.read_csv(train_file_path,index_col='PassengerId')
test_df = pd.read_csv(test_file_path,index_col='PassengerId')

In [ ]:
#33個功能中，survived視輸出標籤，32個特徵
train_df.info()

In [ ]:
#418個測試項目
test_df.info()

# Data Preperation

In [ ]:
#將訓練數據分成inpute feature and output feature
X=train_df.loc[:,'Age':].values.astype('float')
temp=train_df['Survived'].values
print(temp.shape)
y=train_df['Survived'].ravel()

In [ ]:
print(X.shape,y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
#average survival in train and test
#38趴的存活機，62趴的死往機率，存在不平恆
#如果差距太大需要採用不同的方法來建構，所以要常檢查平衡
print('mean survival in train :{0:.3f}'.format(np.mean(y_train)))
print('mean survival in test :{0:.3f}'.format(np.mean(y_test)))

# check scikit-learn version

In [ ]:
import sklearn
sklearn.__version__

# Baseline Model

In [ ]:
from sklearn.dummy import DummyClassifier
#create model，策略指定most_frequent
model_dummy=DummyClassifier(strategy='most_frequent',random_state=0)

In [ ]:
model_dummy.fit(X_train,y_train)

In [ ]:
print('score for baseline model : {0:.2f}'.format(model_dummy.score(X_test,y_test)))

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

In [ ]:
print('accuracy for baseline model: {0:.2f}'.format(accuracy_score(y_test,model_dummy.predict(X_test))))

In [ ]:
print('confusion matrix for baseline model: {0}'.format(confusion_matrix(y_test,model_dummy.predict(X_test))))

In [ ]:
print('precision for baseline model : {0:.2f}'.format(precision_score(y_test,model_dummy.predict(X_test))))
print('recall_score for baseline model : {0:.2f}'.format(recall_score(y_test,model_dummy.predict(X_test))))

# Making first Kaggle submission

In [ ]:
test_X=test_df.values.astype('float')

In [ ]:
#因為這個練習資料沒有真正的labels，必須要提交我們的預測到kaggle
#讓kaggle評估
predictions= model_dummy.predict(test_X)
print(predictions)

In [ ]:
df_submission=pd.DataFrame({'PassengerId':test_df.index,'Survived':predictions})

In [ ]:
df_submission

In [ ]:
submission_data_path = os.path.join(os.path.pardir,'data','external')
submission_file_path = os.path.join(submission_data_path,'01_dummy.csv')

In [ ]:
df_submission.to_csv(submission_file_path,index=False)

In [ ]:
def get_submission_file(model,filename):
    test_X=test_df.values.astype('float')
    predictions= model.predict(test_X)
    df_submission=pd.DataFrame({'PassengerId':test_df.index,'Survived':predictions})
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path,filename)
    df_submission.to_csv(submission_file_path,index=False)
    

In [ ]:
get_submission_file(model_dummy,'01_dummy.csv')

# Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_lr_l=LogisticRegression(random_state=0)

In [ ]:
model_lr_l.fit(X_train,y_train)

In [ ]:
print('Score for logistic regression -version 1: {0:.2f}'.format(model_lr_l.score(X_test,y_test)))

In [ ]:
print('accuracy for logistic regression model: {0:.2f}'.format(accuracy_score(y_test,model_lr_l.predict(X_test))))
print('precision for logistic regression model : {0:.2f}'.format(precision_score(y_test,model_lr_l.predict(X_test))))
print('recall_score for logistic regression model : {0:.2f}'.format(recall_score(y_test,model_lr_l.predict(X_test))))

In [ ]:
#model coefficients，模型權重，模型參數
model_lr_l.coef_

In [ ]:
get_submission_file(model_lr_l,'02_lr.csv')

# Hyperparameter Optimization

In [ ]:
model_lr = LogisticRegression(random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#參數字典
parameters = {'C':[1.0,10.0,50.0,100.0,1000.0],'penalty':['l1','l2']}
#cv=3，執行3被交叉驗證
clf = GridSearchCV(model_lr,param_grid=parameters,cv=3)

In [43]:
clf.fit(X_train,y_train)

D:\ANACONDA\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
D:\ANACONDA\envs\ML\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\ANACONDA\envs\ML\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATI

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [44]:
clf.best_params_

{'C': 1.0, 'penalty': 'l2'}

In [45]:
print('best score :{0:.2f}'.format(clf.best_score_))

best score :0.83


In [ ]:
print('score for logistic regression - version 2: {0:.2f}'.format(clf.score(X_test,y_test)))

# Making Third Submission

In [ ]:
get_submission_file(clf,'03_lr.csv')

# Feature Normalization and Standardization

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
#feature  normalization
scaler= MinMaxScaler()
print(X_train)
X_train_scaled = scaler.fit_transform(X_train)
print(X_train_scaled)

In [36]:
X_train_scaled[:,0].min(),X_train_scaled[:,0].max()

(0.0, 1.0)

In [37]:
X_test_scale=scaler.fit_transform(X_test)

# Feature Standardization

In [40]:
scaler = StandardScaler()
X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled= scaler.transform(X_test)
print(X_train_scaled)

[[ 0.42963418 -0.33167904  3.29646275 ... -1.62827579  0.39477102
  -0.39477102]
 [ 0.12976942 -0.42640542 -0.30335547 ...  0.61414657  0.39477102
  -0.39477102]
 [ 0.12976942  0.10261958 -0.30335547 ... -1.62827579  0.39477102
  -0.39477102]
 ...
 [-0.02016296 -0.48162887 -0.30335547 ... -1.62827579  0.39477102
  -0.39477102]
 [ 0.50460037 -0.28868112 -0.30335547 ...  0.61414657  0.39477102
  -0.39477102]
 [ 2.30378891  0.14245584 -0.30335547 ...  0.61414657  0.39477102
  -0.39477102]]


# Create model after standardization

In [41]:
model_lr=LogisticRegression()
#參數字典
parameters = {'C':[1.0,10.0,50.0,100.0,1000.0],'penalty':['l1','l2']}
#cv=3，執行3被交叉驗證
clf = GridSearchCV(model_lr,param_grid=parameters,cv=3)
clf.fit(X_train_scaled,y_train)


D:\ANACONDA\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
D:\ANACONDA\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
D:\ANACONDA\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
D:\ANACONDA\envs\ML\lib\site-packages\sklearn\model_select

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [42]:
clf.best_score_

0.8118048907326644

In [46]:
print('score for logistic regression - version 2: {0:.2f}'.format(clf.score(X_test_scaled,y_test)))

score for logistic regression - version 2: 0.79


# Model Persistence

In [4]:
import pickle

In [2]:
import os
model_file_path=os.path.join(os.path.pardir,'models','lr_model.pkl')
scaler_file_path=os.path.join(os.path.pardir,'models','lr_scaler.pk1')

In [51]:
#可以用二禁制格式編寫
model_file_pickle=open(model_file_path,'wb')
scaler_file_pickle=open(scaler_file_path,'wb')

In [52]:
pickle.dump(clf,model_file_pickle)
pickle.dump(scaler,scaler_file_pickle)

In [53]:
model_file_pickle.close()

In [54]:
scaler_file_pickle.close()

# load the persisted file

In [7]:

print(model_file_path)
print(scaler_file_path)
with open(model_file_path, 'rb') as f:
    clf_loaded= pickle.load(f)
    print(clf_loaded)
with open(scaler_file_path, 'rb') as f:
    scaler_loaded=pickle.load(f)
#model_file_pickle = open(model_file_path,'r',encoding="utf-8")
#scaler_file_pickle=open(scaler_file_path,'r',encoding="utf-8")
#clf_loaded= pickle.load(model_file_pickle)
#scaler_loaded=pickle.load(scaler_file_pickle)
#model_file_pickle.close()
#scaler_file_pickle.close()

..\models\lr_model.pkl
..\models\lr_scaler.pk1
GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)


In [6]:
clf_loaded

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [62]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [64]:
X_test_scaled= scaler_loaded.transform(X_test)

In [65]:
print('score for peristed logistic regression : {0:.2f}'.format(clf_loaded.score(X_test_scaled,y_test)))

score for peristed logistic regression : 0.79
